# find tracerRNA

example id:
8774__OBKKDPJE_00146__BritoIL_2016__M2.46.ST__bin.47;length1073;#sequences1 

In [1]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/"




In [2]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [3]:
seqid ="AKIPNPCL_00899"
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  AKIPNPCL_00899
NODE_1451_length_29619_cov_4.50203
CosteaPI_2017
CosteaPI_2017__SID713A009-11-90-0__bin.81
1005
[('.', '21722', '24134')]
----------------------------------------------------------------------


In [13]:
spacers=[]
repeats=[]
repeat_start_pos=[]


cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
samplename = genomename.split("__")[1]

print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")


########################################################################

# get CRISPR spacer and repeat sequence

s3_genomename, s3_dataset=filename_discrepancies.get_originalsamplename_froms3name_of_genome(genomename,dataset)
mincedCRISPRfilename=datadir+"1crisprsearch/out/"+s3_dataset+"/"+s3_genomename+".fa.minced.out"
f=open(mincedCRISPRfilename, "r")

is_the_right_contig=False
for line in f.readlines():
    if line.startswith("Sequence \'"): #identifiy line of contigname
        if line.startswith("Sequence \'"+contigname): #check which contig we're on (if there is more than one)
            is_the_right_contig=True
        else:
            is_the_right_contig=False
    if is_the_right_contig:
        print(line)
        if line[0]== "1" or line[0]== "2" or line[0]== "3" or line[0]== "4" or line[0]== "5" or \
        line[0]== "6" or line[0]== "7" or line[0]== "8" or line[0]== "9" or line[0]== "0":

            repeats.append(line.split("\t")[2])

            repeat_start_pos.append(line.split("\t")[0])

            if not line.split("\t")[3]=="\n":
                spacers.append(line.split("\t")[3])
f.close()

# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai seqid della cas e ti dà tt il locus

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.
cas_position={feature:[int(n) for n in cas_dataset[cas_dataset["Seq ID"]==seqid]["Pos"].iloc[0].split()],\
              "Cas1":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas1"].iloc[0])[0][1:]],\
              "Cas2":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas2"].iloc[0])[0][1:]],\
              "CRISPR":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])[0][1:]]
             }
print("Locus features and positions: ", cas_position)
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così
# FALSO! è cosi in cas_dataset, non in tabellazzza! mi ero confuso! trovato il bug!

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("+"*80)
print(cas_position)
print("Retrieving locus")


# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, "/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb", "fasta")
        print("CRISPR si trova", cas_position["CRISPR"])
        print("Cas9 si trova", cas_position[feature])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        temp_alt_cas9start=98503
        temp_alt_cas9stop=101700
        cas9start=cas_position[feature][0]
        cas9stop=cas_position[feature][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()

  
        print(len(cas9_nnseq)/3,len(cas9_aa))


        if my_translated_cas9==cas9_aa:
            print("Cas locus on plus strand")
            plus=True
        if not my_translated_cas9==cas9_aa:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)
        print("+"*80)
        print(cas9_aa)
        
        alignments = pairwise2.align.localxx(my_translated_cas9,cas9_aa)
        print(pairwise2.format_alignment(*alignments[0]))

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        blast_folder="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"
        os.chdir(blast_folder)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")
        unique_repeats=np.unique(repeats)
        for n, repeat in enumerate([Seq(sequence) for sequence in np.unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db
        print("Running blastn of query file agianst all contigs")

        #3. make db file
        dbfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb"
        os.system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")



        blastoutfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11 -penalty -2"
        print(blastn_command)
        os.system(blastn_command)
       # os.system("rm /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracerRNA/temp*")



-----------------------------------------------------
Cas9 id:	AKIPNPCL_00899
SGB:	9340
Genome Name:	 CosteaPI_2017__SID713A009-11-90-0__bin.81
Contig:	NODE_1451_length_29619_cov_4.50203 
-----------------------------------------------------

Sequence 'NODE_1451_length_29619_cov_4.50203' (29619 bp)



CRISPR 1   Range: 21722 - 24134

POSITION	REPEAT				SPACER

--------	------------------------------------	------------------------------

21722		ATTGTAACGTTCTTTCCAAATCAGATATGCTATAAT	GTCTATTCGGCGGCAATTACCGGCAATACG	[ 36, 30 ]

21788		GTTATAGCTTCCTTTCCAAATCAGATATGCTATAAT	GAACCGGCTTGTCGGCAGACAGGGCAAGCG	[ 36, 30 ]

21854		GTTATAGCTTCCTTTCCAAATCAGATATGCTATAAT	CGATAGCTTCGTATAAGGAGACAAACATGA	[ 36, 30 ]

21920		GTTATAGCTTCCTTTCCAAATCAGATATGCTATAAT	TTCTTCCTGCTTTGGTTAAGGTATAGTCAA	[ 36, 30 ]

21986		GTTATAGCTTCCTTTCCAAATCAGATATGCTATAAT	GCCCCGTGACGGCAAGCGCTATACTTTGAG	[ 36, 30 ]

22052		GTTATAGCTTCCTTTCCAAATCAGATATGCTATAAT	CGATAGCTTCGTATAAGGAGACAAACATGA	[ 36, 30 ]

22118		GTTATAGCTTCCTTTCCAAATCAGATATG

In [5]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['21722', '21788', '21854', '21920', '21986', '22052', '22118', '22184', '22250', '22316', '22382', '22448', '22514', '22580', '22646', '22712', '22778', '22844', '22910', '22976', '23042', '23108', '23174', '23240', '23306', '23372', '23438', '23504', '23570', '23636', '23702', '23769', '23835', '23901', '23967', '24033', '24099'] :
-----------------------------------------------------

rpt1|NODE_1451_length_29619_cov_4.50203|CosteaPI_2017__SID713A009-11-90-0__bin.81|AKIPNPCL_00899	NODE_1451_length_29619_cov_4.50203	100.00	36	25	0	0	CTTTCCAAATCAGATATGCTATAAT	CTTTCCAAATCAGATATGCTATAAT	21799	21823	4e-09	plus

rpt1|NODE_1451_length_29619_cov_4.50203|CosteaPI_2017__SID713A009-11-90-0__bin.81|AKIPNPCL_00899	NODE_1451_length_29619_cov_4.50203	100.00	36	25	0	0	CTTTCCAAATCAGATATGCTATAAT	CTTTCCAAATCAGATATGCTATAAT	21865	21889	4e-09	plus

rpt1|NODE_1

## extract sequences

In [10]:
matched_rep_start=29558 #trovato sul minus strand...
matched_rep_stop=29584 

In [14]:
# this is the sequence matching the repeat:
matched_rep=Seq("TTATAGCTTCCTTTCCAAATCAGATAT") #dropped one gap

#rpts which match:
rpt1=Seq(unique_repeats[0])
rpt2=Seq(unique_repeats[1])

# since repeat is on the + strand, and this is all on the + strand, tracrRNA should be its reverse complement:
antirepeat=matched_rep.reverse_complement()
#this should go frombo to bo  = x nt  (x+1)
print(antirepeat)
len(antirepeat)

ATATCTGATTTGGAAAGGAAGCTATAA


27

In [15]:
# print terminators

f=open(blast_folder+str(SGB)+"__"+seqid+"__"+genomename+".terminators")
for line in f.readlines():
    if line.strip().startswith("2"):
        print(line)
f.close()

 2894  Rnamotif + TGCTGAAAAAACTGATGACTTCCGTCGGTTTTGAAACTAT -5.60

20646  Rnamotif - TCCAAAACGTCGGCGATGGATTTGCCcTTATATTTGACT -6.00

21411  Rnamotif - TTGCTCAAGTTGCTTTTTTTCTGAAAGAGCTTTTTCGATTCC -5.80

23468      Both - TATAACCGTACCGTGAGATATAGCTCGCGcTCTTATATTATA -7.40

24059      Both - AACAAAAAAAAGCACAGTAATATACTGTGCTTTATTATAGCAT -10.50

24061      Both + GCTATAATAAAGCACAGTATATTACTGTGCTTTTTTTTGTTA -10.50

24135      Both - AAAACAAAAAAGCCGTTGATATTCAACGGCTTAATTTGAATCT -11.80

24137      Both + ATTCAAATTAAGCCGTTGAATATCAACGGCTTTTTTGTTTTT -11.80

24950      Both - AAAATTCGGAAGCTTTTGCCGCCGGAATTTACTGGAAAAGCTTTTTTACGGATT NA

26355  Rnamotif - GTTATGTCAAACGGTATTTGGAAGATGCCGTTGATTTTTCCG -7.40

27839  Rnamotif + TAAACGGCTACGCCCTTGTTTGGGCcTTTTTGCGTCGT -7.80



In [7]:
# this is the rho-independent trnascription terminator,

terminator_end=27839-1 # (sarebe 1463-1, for indexing reasons) sarebbe la fine xke prendiamo il rev comp remembra. che ha senso xke l'antirepeat è a 14729, che è dopo
 #oppure, ce n'è uno con TTTTTTT a 14653...
terminator=Seq("TAAACGGCTACGCCCTTGTTTGGGCcTTTTTGCGTCGT".upper()).reverse_complement() # un po' torna che il blast sia sul - e questo sia sul + , solo xke di solito era il contrario. xrò è trpp lungo.. # da aggiuntere .reverse_complement() se lo strand indicato è + ?
print(len(terminator))
terminator_start=terminator_end+len(terminator) #occhio che qui start e stop si riferisce al rev.compl, mentre nel antirepeat si riferisce alla direzione base

38


In [16]:
#se terminator on + strand:
terminator_start=27839#18895-1
terminator=Seq("TAAACGGCTACGCCCTTGTTTGGGCcTTTTTGCGTCGT".upper())
print("terminator length: ", len(terminator))
terminator_end=terminator_start+len(terminator)

terminator length:  38


In [18]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:
for record in SeqIO.parse("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/selected_loci/"+seqid+"/"+genomename+".fa", "fasta"):
    if record.id==contigname:
        contigseq=record.seq

#check antirepeat
print("test repeat: ", antirepeat, len(antirepeat), contigseq[matched_rep_start-1:matched_rep_stop]==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb # ho levato il .rev_compl xke è tt su questostrand il tracrRNA
print(contigseq[matched_rep_start-1:matched_rep_stop])
# check terminator
print("test terminator: " ,terminator, len(terminator), contigseq[terminator_end:terminator_start].reverse_complement()==terminator)

pre_tracrRNA=contigseq[matched_rep_start:terminator_end]
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario=contigseq[terminator_start:matched_rep_start-1].reverse_complement()
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

test repeat:  ATATCTGATTTGGAAAGGAAGCTATAA 27 True
ATATCTGATTTGGAAAGGAAGCTATAA
test terminator:  TAAACGGCTACGCCCTTGTTTGGGCCTTTTTGCGTCGT 38 False
pre tracrRNA:  0
mario sequence: AAAATCAAAAAAGTTTCTTTATCAAAAGAATATTACGGAATGTAGTTTCCGTCAAGTGTAATTTTTACTTTCGGAATAAGACGCCTAGGATTGCGGTTTTCCCGGACAGTGAAAGAAAAAGCGCAAAACTGCCGTTTTTAAGAAAGTCCGTTTTAAGGTTGAAAATGTTTTTTTATGTATCGGTGCAGCGTACTCCAGCTGCATTCGAATTCCCGGGCGATTTCCGCTTGTGAAATGTTGCGGGCCAGCATCCGTTCGATCTGCCGGTGATGTTTTGCCAGTTTTTGGTAATTGAAACCATAAGGGCGGCCCAAGTGTCGTCCTTCGGATTTCAATCGTTTGAGAGATTCTCTGGTCCTTTGCGAAATCAACTGGCGTTCAATTTCCCCGGACAGGGAAAAGGCAAAAGCGAGGACTTTTGATTGGATGTCGGCTCCCAGTCGGTAATTTTCCTTCAATGTCCAAATTTGGCATTCTTTTTCCAGACATTTTTGCAAAATGCCCATCACTTCCAACATGTTGCGTCCAAGTCTTGATATTTCGGTGGCAACGATGACGTCTCCCGCTTTTATCTTTTGCAGGAGTTTTCCCAGTTTGCGCTGTTTCAGCTCTTTGCGCGACGAAATGGTTTCCGTGATCCAGGCGTCGATTTTAAGGTGCTGCCGCTTTGAAAATTCTTTAATTTCGTGGTGTTGGTTTTCGATGTTTTGATGGTCGGTTGAAACGCGCACATATCCATATATCATGATCAGTTCCTCGTAAAAAGTTTTTAACTCGGAAAGACCATATCATTTAAAAGAGCGTTTATTTTCTTCACTTAAAA

hmmm

In [22]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch

alignments = pairwise2.align.localxx(rpt1, antirepeat.reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt1))


GCCT-TG-AGCACGCA-CCCTTCCCCAAGGTGATACGCT
|  | || ||| |||  |||||||||||||||  |
G--TCTGGAGC-CGC-GCCCTTCCCCAAGGTG--A----
  Score=26

rpt length: 36


In [24]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx((mario).reverse_complement(), rpt1)
print(pairwise2.format_alignment(*alignments[0]))

AAGCCTCAAAAAAAGAAGACTGATTCGACCTCAGATCTACATCTGCGATTCATTTTTTTCGAGACGAAATGCGCCGGCACGTCGCGCTTCACGCAAGAAAAGAGCCGTATTCGGCACAGAGATTTATGTACTTTTCTTCACAATCGAATCAATCCTTAGGCATAAAATTGCTCAAAAGGACCCGCCTAAATGCTATTCATAGCTACCGTTTACGGAATACTGGCTAACACTCGCCCAAAAAAGTTCGCCCCGCCCAATCCGAAGATTGAACGGGGGTTACTTTACGGGTAAGCCACTTAAGTGGCTCCCCTT
                                                                                                                                                                                       | |      |   |  | |  |  |    |     ||  |    ||    |||       ||     | |||||   |  |  |        |       |    |     ||       |     | |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------G-C------C---T--T-G--A--G----C-----AC--G----CA----CCC-------TT-----C-CCCAA---G--G--T--------G-------A----T-----AC-------G-----C-T
  Score=36



In [25]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

print("\n----------------------------------------------------------------------")
print("         tracrRNA sequence for", feature,"protein id", seqid)
print("----------------------------------------------------------------------\n")
print("- contig name:", cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print("- Dataset: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print("- Bin: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print("- SGB: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print("----------------------------------------------------------------------\n")
print("- Length: ", len(tracrRNA),"\n- position: ", terminator_end,matched_rep_stop)
print(tracrRNA)


----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id LJKGNKPJ_00564
----------------------------------------------------------------------

- contig name: NODE_211_length_45519_cov_7.71687
- Dataset:  KarlssonFH_2013
- Bin:  KarlssonFH_2013__S420__bin.8
- SGB:  14454
----------------------------------------------------------------------

- Length:  381 
- position:  13889 14270
TCACCTTGGGGAAGGGCGCGGCTCCAGACAAGGGGAGCCACTTAAGTGGCTTACCCGTAAAGTAACCCCCGTTCAATCTTCGGATTGGGCGGGGCGAACTTTTTTGGGCGAGTGTTAGCCAGTATTCCGTAAACGGTAGCTATGAATAGCATTTAGGCGGGTCCTTTTGAGCAATTTTATGCCTAAGGATTGATTCGATTGTGAAGAAAAGTACATAAATCTCTGTGCCGAATACGGCTCTTTTCTTGCGTGAAGCGCGACGTGCCGGCGCATTTCGTCTCGAAAAAAATGAATCGCAGATGTAGATCTGAGGTCGAATCAGTCTTCTTTTTTTGAGGCTTGAAACAGAAAAGCCCCTGTTAAAGGGGCTTTTCTGTGCGC
